# From Decision Trees to Random Forests

```
Authors: Alexandre Gramfort
         Thomas Moreau
```

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [1]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [2]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [3]:
bootstrap_idx(X_train)

array([ 27,  14,   1, 111,  74,   9,  28,  34,  72,  83, 104,  15, 103,
        85,  18,   3,  67,   7,  76,  26,  91,  20,  17,  17,  29,  76,
        75,  81,  13,  26,   0,  14,   2,  47,  41,  78,  44, 106, 106,
        90,  56,  50,   4,  15,  69,  27,  26,   0,  68,  18,   7,  73,
        31,  88, 108,  11,  81,  32,  79, 104,  49,  50,  49,  92,  20,
        44,  27,  46,  39,  86,   2,  97, 106,   7,  48,  31,  42,  36,
       107,  64,  82,  79,  85,   6,  82, 111,  74,  33,  83,  18,  56,
       100,  80,   5, 109,   2,  52,  17,  27,  42, 108,  18, 105,  96,
        66,  81, 108,  58,  81,  95,  97,   3])

In [4]:
from collections import Counter
Counter(bootstrap_idx(X_train))

Counter({59: 1,
         93: 3,
         36: 1,
         110: 1,
         84: 3,
         32: 1,
         46: 1,
         78: 2,
         11: 2,
         86: 3,
         6: 2,
         79: 3,
         16: 1,
         90: 1,
         64: 1,
         83: 1,
         105: 3,
         10: 2,
         3: 2,
         65: 3,
         108: 1,
         70: 2,
         89: 1,
         73: 2,
         53: 1,
         100: 3,
         24: 1,
         94: 1,
         56: 2,
         52: 3,
         74: 2,
         34: 1,
         28: 3,
         106: 1,
         25: 2,
         50: 1,
         15: 1,
         58: 2,
         104: 1,
         101: 2,
         85: 2,
         109: 1,
         12: 1,
         102: 1,
         95: 2,
         2: 2,
         87: 1,
         37: 1,
         0: 2,
         8: 3,
         44: 1,
         69: 1,
         92: 2,
         5: 1,
         42: 2,
         63: 3,
         40: 1,
         72: 3,
         38: 1,
         39: 1,
         111: 1,
         91: 1,
    

In [5]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [6]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [7]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

Classes distribution in the original data: Counter({0: 38, 1: 37, 2: 37})
Classes distribution in the bootstrap: Counter({2: 41, 1: 37, 0: 34})


In [ ]:
#import ipdb
#ipdb.set_trace()

--Call--
> /Users/pierreloviton/opt/anaconda3/lib/python3.9/site-packages/IPython/core/displayhook.py(252)__call__()
    251 
--> 252     def __call__(self, result=None):
    253         """Printing with history cache management.



<div class="alert alert-success">
    <b>EXERCISE: Create a bagging classifier</b>:<br>
    <br>
    A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.
     <ul>
      <li>
      Create several <code>DecisionTreeClassifier</code> and store them in a Python list;
      </li>
      <li>
      Loop over these trees and <code>fit</code> them by generating a bootstrap sample using <code>bootstrap_sample</code> function;
      </li>
      <li>
      To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the <code>predict</code> method. Aggregate all predictions in a NumPy array;
      </li>
      <li>
      Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
      </li>
      <li>
      Finally, check the accuracy of your model.
      </li>
    </ul>
</div>

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample
import numpy as np

# Number of decision trees in the ensemble
n_estimators = 10

# Create a list to store the decision trees
trees = []

# Create the decision trees
for i in range(n_estimators):
    trees.append(DecisionTreeClassifier())

# Bootstrap sample the training data
X_train, y_train = resample(X_train, y_train)

# Fit the decision trees on the bootstrapped data
for tree in trees:
    tree.fit(X_train, y_train)

# Make predictions on the test set
predictions = []
for tree in trees:
    predictions.append(tree.predict(X_test))

# Convert the list of arrays into a 2D array
predictions = np.array(predictions)

# Take the majority vote of the predictions
final_predictions = []
for i in range(len(X_test)):
    final_predictions.append(np.argmax(np.bincount(predictions[:,i])))

# Calculate the accuracy of the model
accuracy = sum(final_predictions == y_test) / len(y_test)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [23]:
from sklearn.tree import DecisionTreeClassifier
from statistics import mode
list_ = [DecisionTreeClassifier() for _ in range(3)]
preds = []
for tree in list_:
    tree.fit(*bootstrap_sample(X, y))
    preds.append(tree.predict(X))
pred_ = np.array([mode(np.stack(preds)[:, i]) for i in range(y.size)])
np.mean(pred_==y)

0.9933333333333333

In [13]:
from sklearn.tree import DecisionTreeClassifier
from statistics import mode

list_tree = [DecisionTreeClassifier() for i in range(10)]

pred_array = []
for tree in list_tree:
    X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)
    idx_features = np.random.choice(X.shape[1], size=int(np.sqrt(X.shape[1])), replace=False)
    X_train_bootstrap = X_train_bootstrap[:, idx_features]
    tree.fit(X_train_bootstrap, y_train_bootstrap)

    pred = tree.predict(X_test[:, idx_features])
    pred_array.append(pred)

pred_array = np.asarray(pred_array)
y_pred = np.mode(pred_array, axis=0).mode[0]
(y_pred == y_test).mean()

AttributeError: module 'numpy' has no attribute 'mode'

<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own bagging classifier, use a <code>BaggingClassifier</code> from scikit-learn to fit the above data.
</div>

In [28]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Number of decision trees in the ensemble
n_estimators = 10

# Create the bagging classifier
bagging_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=n_estimators, bootstrap=True)

# Fit the classifier on the training data
bagging_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = bagging_clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = sum(y_pred == y_test) / len(y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9473684210526315


## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

<div class="alert alert-success">
    <b>EXERCISE: Create a random forest classifier</b>:
    <br>
    Use your previous code which was generated several <code>DecisionTreeClassifier</code>. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.
</div>

In [29]:
from math import sqrt
from sklearn.ensemble import RandomForestClassifier

# Number of decision trees in the ensemble
n_estimators = 10

# Number of features in the dataset
n_features = X_train.shape[1]

# The number of features to consider when looking for the best split
max_features = int(sqrt(n_features))

# Create the random forest classifier
rf_clf = RandomForestClassifier(n_estimators=n_estimators, max_features=max_features)

# Fit the classifier on the training data
rf_clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_clf.predict(X_test)

# Calculate the accuracy of the model
accuracy = sum(y_pred == y_test) / len(y_test)
print("Accuracy:", accuracy)


Accuracy: 0.9736842105263158


<div class="alert alert-success">
    <b>EXERCISE: using scikit-learn</b>:
    <br>
    After implementing your own random forest classifier, use a <code>RandomForestClassifier</code> from scikit-learn to fit the above data.
</div>

In [30]:
from figures import plot_forest_interactive
plot_forest_interactive()

interactive(children=(IntSlider(value=0, description='max_depth', max=8), Output()), _dom_classes=('widget-int…